<a href="https://colab.research.google.com/github/yuri200819/-/blob/main/%E6%96%87%E7%AB%A0%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

def count_codePoint(str):
    counter = np.zeros(65535)
    for i in range(len(str)):
        code_point = ord(str[i])
        if code_point > 65535 :
            countinue
        counter[code_point] += 1

    counter = counter/len(str)
    return counter
#面倒なのでとりあえず日本語と英語のみ
ja_str = "これは日本語の文章です。"
en_str = "This is English Sentences."

x_train = [count_codePoint(ja_str),count_codePoint(en_str)]
y_train = ["ja","en"]

clf = GaussianNB()
clf.fit(x_train, y_train)

ja_test_str = "こんにちは"
en_test_str = "Hello"

x_test = [count_codePoint(en_test_str),count_codePoint(ja_test_str)]
y_test = ["en","ja"]

y_pred = clf.predict(x_test)
print(y_pred)
print("正解率 = ", accuracy_score(y_test, y_pred))

['en' 'ja']
正解率 =  1.0


In [ ]:
#WindowsでMECABは面倒。ColaboratoryってUbuntuで動いているみたいですね。
#だったら、UbuntuにMeCabを入れる要領でやれば、使えるでしょって思ったのでやってみました。という記事見つけたので
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.7)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.10)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.4)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+ma

In [ ]:
import MeCab
import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
m=MeCab.Tagger("-d {0}".format(path))

In [ ]:
import MeCab
tagger = MeCab.Tagger()
result = tagger.parse("メイが恋ダンスを踊っている。")
print(result)

メイ	名詞,一般,*,*,*,*,*
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
恋	名詞,一般,*,*,*,*,恋,コイ,コイ
ダンス	名詞,サ変接続,*,*,*,*,ダンス,ダンス,ダンス
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
踊っ	動詞,自立,*,*,五段・ラ行,連用タ接続,踊る,オドッ,オドッ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
いる	動詞,非自立,*,*,一段,基本形,いる,イル,イル
。	記号,句点,*,*,*,*,。,。,。
EOS



In [ ]:
#文章分類
import zipfile
import os.path
import urllib.request as req
import MeCab
from gensim import models
from gensim.models.doc2vec import TaggedDocument

mecab = MeCab.Tagger()
mecab.parse("")

#面倒なのでとりあえず宮沢賢治１作品で試作
list = [
    {"auther":{
        "name":"宮沢賢治",
        "url":"https://www.aozora.gr.jp/cards/000081/files/"},
     "books":[
        {"name":"銀河鉄道の夜","zipname":"43737_ruby_19028.zip"},
     ]},
]

def book_list():
    for novelist in list:
        auther = novelist["auther"]
        for book in novelist["books"]:
            yield auther, book

def read_book(auther, book):
    zipname = book["zipname"]
    if not os.path.exists(zipname):
        req.urlretrieve(auther["url"] + zipname,zipname)
    zipname = book["zipname"]

    with zipfile.ZipFile(zipname,"r") as zf:
        for filename in zf.namelist():
            if os.path.splitext(filename)[1] != ".txt":
                continue
            with zf.open(filename,"r") as f:
                return f.read().decode("shift-jis")


def split_words(text):
    node = mecab.parseToNode(text)
    wakati_words =[]
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if hinshi in ["名詞"]:
            wakati_words.append(node.surface)
        elif hinshi in ["動詞","形容詞"]:
            wakati_words.append(node.feature.split(",")[6])
        node = node.next
    return wakati_words
documents = []
for auther, book in book_list():
    words = read_book(auther, book)
    wakati_words = split_words(words)
    document = TaggedDocument(
        wakati_words, [auther["name"] + ":" + book["name"]])
    documents.append(document)

model = models.Doc2Vec(
    documents, dm=0, vector_size=300, window=15, min_count=1)
model.save("aozora.model")
print("モデル作成終了")

モデル作成終了


In [ ]:
#作者を分類してみる
import urllib.request as req
import zipfile
import os.path
import MeCab
from gensim import models

mecab = MeCab.Tagger()
mecab.parse("")

model = models.Doc2Vec.load("aozora.model")

def read_book(url, zipname):
    if not os.path.exists(zipname):
        req.urlretrieve(url, zipname)

    with zipfile.ZipFile(zipname,"r") as zf:
        for filename in zf.namelist():
            with zf.open(filename, "r") as f:
                return f.read().decode("shift-jis")

def split_words(text):
    node = mecab.parseToNode(text)
    wakati_words = []
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if hinshi in ["名詞"]:
           wakati_words.append(node.surface)
        elif hinshi in ["動詞","形容詞"]:
            wakati_words.append(node.feature.split(",")[6])
        node = node.next
    return wakati_words

def similar(title, url):
    zipname = url.split("/")[-1]

    words = read_book(url, zipname)
    wakati_words = split_words(words)
    vector = model.infer_vector(wakati_words)
    print("--- 「" + title + "」と似た作品は？---")
    print(model.docvecs.most_similar([vector],topn=3))
    print("")

similar("宮沢賢治：よだかの星",
        "https://www.aozora.gr.jp/cards/000081/files/473_ruby_467.zip")

--- 「宮沢賢治：よだかの星」と似た作品は？---
[('宮沢賢治:銀河鉄道の夜', 0.9999933838844299)]



/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
#自動作文に挑戦。。。しようとしたけどmecab-ipadic-neologdがうまくいかないのかな？
import MeCab
import os,json,random

dict_file = "markov_dict.json"
dic = {}

def regist_dic(wordlist):
    global dic
    w1 = ""
    w2 = ""

    if len(wordlist) < 3 : return

    for w in wordlist :
        word = w[0]
        if word =="" or word == "\r\n" or word == "\n" : continue
        if w1 and w2 :
            set_dic(dic,w1, w2, word)
        if word == "。" or word == "?" or word == "？" :
            w1 = ""
            w2 = ""
            continue
        w1, w2 = w2, word

    json.dump(dic, open(dict_file,"w", encoding="utf-8"))

def set_dic(dic, w1, w2, w3):
    if w1 not in dic : dic[w1] = {}
    if w2 not in dic[w1] : dic[w1][w2] = {}
    if w3 not in dic[w1][w2] : dic[w1][w2][w3] = 0
    dic[w1][w2][w3] += 1

def make_response(word):
    res = []

    w1 = word
    res.append(w1)
    w2 = word_choice(dic[w1])
    res.append(w2)
    while True:
        if w1 in dic and w2 in dic[w1] : w3 = word_choice(dic[w1][w2])
        else : w3 = ""
        res.append(w3)
        if w3 =="。"or w3 == word == "?" or word == "？" or w3 =="": break
        w1,w2 = w2,w3
    return "".join(res)

def word_choice(candidate):
    keys = candidate.keys()
    return random.choice(list(keys))

if os.path.exists(dict_file):
        dic = json.load(open(dict_file,"r"))

while True:
    text = input("You _> ")
    if text == "" or text =="さようなら" :
        print("Bot _> さようなら")
        break

    if text[-1] != "。" and text[-1] !="?" and text[-1] != "？" : text +="。"
#形態素解析
    tagger = MeCab.Tagger("-d /var/lib/mecab/dic/mecab-ipadic-neologd")
    tagger.parse("")
    node = tagger.parseToNode(text)

    wordlist = []
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if hinshi not in ["BOS/EOS"]:
           wordlist.append([node.surface,hinshi])
        node = node.next

    regist_dic(wordlist)

    for w in wordlist:
        word = w[0]
        hinshi = w[1]

        if hinshi in ["感動詞"] :
            print("Bot _> " + word)
            break
        elif (hinshi in ["名詞" ,"形容詞" ,"動詞" ]) and (word in dic):
            print("Bot _> " + make_response(word))
            break

You _> さようなら
Bot _> さようなら


In [ ]:
#colaboratoryでmecab-ipadic-neologdは使えないのか！！